In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from scipy.interpolate import make_interp_spline

import geopandas as gpd
from shapely.geometry import Polygon, Point
from shapely.ops import transform
import mercantile
import pyproj
from ipyleaflet import Map, DrawControl

import requests
from io import StringIO
import json
from tqdm import tqdm

import ipywidgets as widgets

from datetime import datetime, timedelta

import re
from functools import partial

import os


# Decide the zone of the data we will treat (example City Zürich or many cities)

## Upload a csv with column Name and Google Maps Link of the place or enter it manually

In [ ]:
Places #=insert csv where you have the data 

In [ ]:
Places = {'Name': ['Zurich'
                   #,'Name_of_city_2'
                   #,...
                  ]
          , 'Google Maps Link': ['https://www.google.com/maps/@47.3775328,8.5362255,21z?entry=ttu'
                                 #,'Google_maps_link_2'
                                 #,...
                                ]}
Places = pd.DataFrame(Places)

## Define the radius to apply

In [ ]:
#Insert radius in meters
radius_to_apply = 50

## Define the functions

In [ ]:
def coords2points(lat, lon):
    """
    Converts latitude and longitude to a Shapely Point object.

    Args:
        lat (float): Latitude.
        lon (float): Longitude.

    Returns:
        A shapely.geometry.Point object.
    """
    return Point(lon, lat)

def extract_coordinates_from_google_maps_link(google_maps_link):
    """
    Extracts latitude and longitude from a Google Maps link.

    Args:
        google_maps_link (str): Google Maps link.

    Returns:
        Latitude and longitude as floats.
    """
    # Regular expression to extract coordinates from the Google Maps link
    pattern = r'@(-?\d+\.\d+),(-?\d+\.\d+)'
    match = re.search(pattern, google_maps_link)
    if match:
        lat = float(match.group(1))
        lon = float(match.group(2))
        return lat, lon
    else:
        raise ValueError("Invalid Google Maps link. Unable to extract coordinates.")

def aeqd_reproj_buffer(center, radius=radius_to_apply):
    """
    Converts center coordinates to AEQD projection,
    draws a circle of given radius around the center coordinates,
    converts both polygons back to WGS84.

    Args:
        center (shapely.geometry Point): Center coordinates of a circle.
        radius (integer): Circle's radius in meters.

    Returns:
        A shapely.geometry Polygon object for circle of given radius.
    """
    # Get the latitude, longitude of the center coordinates
    lat = center.y
    lon = center.x

    # Define the projections
    local_azimuthal_projection = "+proj=aeqd +R=6371000 +units=m +lat_0={} +lon_0={}".format(
        lat, lon
    )
    wgs84_to_aeqd = partial(
        pyproj.transform,
        pyproj.Proj("+proj=longlat +datum=WGS84 +no_defs"),
        pyproj.Proj(local_azimuthal_projection),
    )
    aeqd_to_wgs84 = partial(
        pyproj.transform,
        pyproj.Proj(local_azimuthal_projection),
        pyproj.Proj("+proj=longlat +datum=WGS84 +no_defs"),
    )

    # Transform the center coordinates from WGS84 to AEQD
    point_transformed = transform(wgs84_to_aeqd, center)
    buffer = point_transformed.buffer(radius)

    # Get the polygon with lat lon coordinates
    circle_poly = transform(aeqd_to_wgs84, buffer)

    return circle_poly


def generate_quadkeys(circle_poly, zoom):
    """
    Generate a list of quadkeys that overlap with the zone.
    
    Args:
        circle_poly (shapely.geometry Polygon): circle polygon object drawn 
            around a place.
        zoom (integer): zoom level.
        
    Return:
        List of quadkeys as string
    """

    return [mercantile.quadkey(x) for x in mercantile.tiles(*circle_poly.bounds, zoom)]


In [ ]:
places_df = pd.DataFrame()
places_df['name'] = Places['Name']
places_df['lat'] = Places.apply(lambda x: extract_coordinates_from_google_maps_link(x['Google Maps Link']),axis=1)
places_df['point'] =  places_df.apply(lambda x: coords2points(x['lat'][0],x['lat'][1]),axis=1)
places_df['aeqd_reproj_circle'] = places_df.apply(lambda x: aeqd_reproj_buffer(x['point']),axis=1)


# Create a list of overlapping z18 quadkeys for each zone and add to a new column
places_df['z18_quadkeys'] = places_df.apply(lambda x: generate_quadkeys(x['aeqd_reproj_circle'], 18),axis=1)
places_df['z7_quadkeys'] = places_df.apply(lambda x: generate_quadkeys(x['aeqd_reproj_circle'], 7),axis=1)

#save the quadkeys
all_z7_quadkeys = list(set([quadkey for sublist in places_df['z7_quadkeys'] for quadkey in sublist]))
all_z18_quadkeys = list(set([quadkey for sublist in places_df['z18_quadkeys'] for quadkey in sublist]))

# Import Data

In [ ]:
import os
import requests
from datetime import datetime, timedelta

def import_sample_data_to_df(start_date, end_date, z7_quadkey_list, verbose=True):

    
    # Generate range of dates between start and end date in %Y-%m-%d string format
    start = datetime.strptime(start_date, '%Y-%m-%d')
    end = datetime.strptime(end_date, '%Y-%m-%d')
    num_days = int((end - start).days)
    days_range = num_days + 1
    date_range = [(start + timedelta(n)).strftime('%Y-%m-%d') for n in range(days_range)]

    sample_data = []
    for z7_quadkey in z7_quadkey_list:
        for i in range(len(date_range)):
            yr, month, date = date_range[i].split('-')
            path = f"D:/non_driving_activity/{z7_quadkey}/{yr}/{month}/{date}/{z7_quadkey}_{month}_{date}.csv"
            df = pd.read_csv(path, sep='|')
            convert_dict = {'geography': str,'xlon': float, 'xlat':float,'bounds':str,'agg_day_period': 'datetime64[ns]', 'activity_index_nondriving': float, } 
            df = df.astype(convert_dict)
            #Keep leading zeros and save as string
            df['z18_quadkey'] = df.apply(lambda x: x['geography'].zfill(18), axis=1).astype('str')
            df['z7_quadkey'] = df.apply(lambda x: x['geography'][:6].zfill(7), axis=1).astype('str')
            # Check if 'z18_quadkey' values are in unique_z18_quadkeys
            df_filtered = df[df['z18_quadkey'].isin(all_z18_quadkeys)]

            # Append DataFrame to sample_data if it meets the condition
            if not df_filtered.empty:
                sample_data.append(df_filtered)

            print(path)
    
    return  pd.concat(sample_data)

In [ ]:
# Tweak the following set of parameters to include broader time frame or more airports
start_dt_str = "2022-12-28"
end_dt_str = "2023-01-05"

# Define a list to append all newly created DataFrames
footfall_zone = []
for z7 in all_z7_quadkeys:
    print ([z7])

    # Run the download script
    footfall_zone.append(import_sample_data_to_df(start_dt_str, end_dt_str, [z7], False))

# Redefine at the z18 Quadkey of the places and export

In [ ]:
data_to_filter = pd.concat(footfall_zone, ignore_index=True)

In [ ]:
data_to_export = data_to_filter[data_to_filter['z18_quadkey'].isin(all_z18_quadkeys)]

In [ ]:
df_filtered.to_csv('Documents/Footfall/stores_2022_footfall.csv', index=False)